# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

---

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2  # 1 would be where you need to specify the files
#%aimport data_fcns

import pandas as pd # dataframes
import os
import gspread_pandas
from gspread_pandas import Spread, Client # gsheets interaction

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

# import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Custom
import data_fcns as dfc
import keys  # Custom keys lib
import comic_recs as cr

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

import time

import numpy as np

In [2]:
# instantiate SparkSession object
spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local").getOrCreate()

jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .load()

# Define path to secret
secret_path_aws = os.path.join(os.environ['HOME'], '.secret', 
                           'aws_ps_flatiron.json')
secret_path_aws

aws_keys = keys.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

## Import the data

There is way to directly hit PostgreSQL through JDBC, but I don't know how to do that yet. So have worked around by saving the candidate dataset to JSON, and then will use that as input to Spark.

In [3]:
trans = spark.read.json('raw_data/trans_filtered.json')

In [4]:
# Persist the data
trans.persist()

DataFrame[account_num: string, comic_title: string, date_sold: bigint, item_id: string, publisher: string, qty_sold: bigint, title_and_num: string]

In [5]:
print(trans.count(), len(trans.columns))

327839 7


In [6]:
# check schema
trans.printSchema()

root
 |-- account_num: string (nullable = true)
 |-- comic_title: string (nullable = true)
 |-- date_sold: long (nullable = true)
 |-- item_id: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- qty_sold: long (nullable = true)
 |-- title_and_num: string (nullable = true)



### More exploration/testing

We won't be using pandas dataframes in the matrix factorization through Spark, but let's cast to one anyway as it will be easier to work with for EDA.

In [7]:
# cast to Pandas dataframe to turn timestamp data to datetime and check nulls. 
trans_df = trans.select('*').toPandas()
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327839 entries, 0 to 327838
Data columns (total 7 columns):
account_num      327839 non-null object
comic_title      327839 non-null object
date_sold        327839 non-null int64
item_id          327839 non-null object
publisher        327839 non-null object
qty_sold         327839 non-null int64
title_and_num    327839 non-null object
dtypes: int64(2), object(5)
memory usage: 17.5+ MB


In [8]:
# Let's double check the data is how we expect it
trans_df.head()

,account_num,comic_title,date_sold,item_id,publisher,qty_sold,title_and_num
0,00399,Royal Historian of Oz (SLG),1279136980000,DCD416182,Amaze Ink Slave Labor Graphics,1,Royal Historian of Oz #1
1,00327,Royal Historian of Oz (SLG),1288543119000,DCD416182,Amaze Ink Slave Labor Graphics,1,Royal Historian of Oz #1
2,00327,Royal Historian of Oz (SLG),1288543119000,DCD423794,Amaze Ink Slave Labor Graphics,1,Royal Historian of Oz #2
3,01065,Warlord of Io & Other Storie (SLG),1412166247000,DCD390709,Amaze Ink Slave Labor Graphics,1,Warlord of Io & Other Stories
4,01033,Afterlife With Archie (Archie),1390505789000,DCD630105,Archie Comics,1,Afterlife With Archie #1 2nd P


In [9]:
trans_df['dt'] = pd.to_datetime(trans_df['date_sold'], unit='ms')

Yes. Reverse-confirmed versus the original transactions dataframe in the other notebook that this datetime is correct. 

### Data Prep for ALS

Let's aggregate the data to the two columns we need:
- `account_num` - This is the identifier for individual customers.


- `comic_title` - The comic. Represents individual volumes/runs of a comic.


- `score` - We need to figure out what we want to use to act as a `score`. If these were Amazon items then review scores would be natural fit; but we don't have that. We can maybe use a binary flag of `bought`/`not bought`. Or we can use the `qty_sold`. This might be interesting in that it might capture some interesting behavior from comic 'collectors/speculators'. Since this is first pass, I'm curious as to what `qty_sold` might do!


We only care about `account_num`, `comic_title` and `qty_sold`.

In [10]:
comics_sold = trans[['account_num', 'comic_title', 'qty_sold']]
comics_sold.persist()

DataFrame[account_num: string, comic_title: string, qty_sold: bigint]

In [11]:
comics_sold = comics_sold.withColumn('bought', lit(1))

In [12]:
comics_sold.show(10)

+-----------+--------------------+--------+------+
|account_num|         comic_title|qty_sold|bought|
+-----------+--------------------+--------+------+
|      00399|Royal Historian o...|       1|     1|
|      00327|Royal Historian o...|       1|     1|
|      00327|Royal Historian o...|       1|     1|
|      01065|Warlord of Io & O...|       1|     1|
|      01033|Afterlife With Ar...|       1|     1|
|      01333|Afterlife With Ar...|       1|     1|
|      00946|Afterlife With Ar...|       1|     1|
|      01278|Afterlife With Ar...|       1|     1|
|      01212|Afterlife With Ar...|       1|     1|
|      00877|Afterlife With Ar...|       1|     1|
+-----------+--------------------+--------+------+
only showing top 10 rows



In [13]:
comics_sold = trans[['account_num', 'comic_title', 'qty_sold']]
comics_sold.persist()

DataFrame[account_num: string, comic_title: string, qty_sold: bigint]

In [14]:
total_comics_sold = comics_sold.groupBy(['account_num', 'comic_title']).agg({'qty_sold':'sum'})
total_comics_sold.persist()

DataFrame[account_num: string, comic_title: string, sum(qty_sold): bigint]

Ok, let's take a look at the results.

In [15]:
total_comics_sold.show(10)

+-----------+--------------------+-------------+
|account_num|         comic_title|sum(qty_sold)|
+-----------+--------------------+-------------+
|      02247|Bubblegun VOL 2 (...|            1|
|      00487|Captain Swing (Av...|            2|
|      00029|God Is Dead (Avatar)|            7|
|      01260| Providence (Avatar)|            1|
|      00172|   Supergod (Avatar)|            3|
|      02493|       Abbott (Boom)|            3|
|      00052|Adventure Time Ma...|            6|
|      00032|Big Trouble In Li...|           11|
|      01149| Broken World (Boom)|            2|
|      01489|Jim Henson Labyri...|            1|
+-----------+--------------------+-------------+
only showing top 10 rows



In [16]:
print(total_comics_sold.count(), len(total_comics_sold.columns))

61871 3


In [17]:
total_comics_sold = total_comics_sold.withColumn('bought', lit(1))

I don't like that default column name. Let's fix that to be `qty_sold` again.

In [18]:
total_comics_sold.show(10)

+-----------+--------------------+-------------+------+
|account_num|         comic_title|sum(qty_sold)|bought|
+-----------+--------------------+-------------+------+
|      02247|Bubblegun VOL 2 (...|            1|     1|
|      00487|Captain Swing (Av...|            2|     1|
|      00029|God Is Dead (Avatar)|            7|     1|
|      01260| Providence (Avatar)|            1|     1|
|      00172|   Supergod (Avatar)|            3|     1|
|      02493|       Abbott (Boom)|            3|     1|
|      00052|Adventure Time Ma...|            6|     1|
|      00032|Big Trouble In Li...|           11|     1|
|      01149| Broken World (Boom)|            2|     1|
|      01489|Jim Henson Labyri...|            1|     1|
+-----------+--------------------+-------------+------+
only showing top 10 rows



In [19]:
total_comics_sold = total_comics_sold[['account_num', 'comic_title', 'bought']]

In [20]:
print(total_comics_sold.count(), len(total_comics_sold.columns))

61871 3


### Formatting

Sooooooo, I forgot that the values need to be numeric. So need to fix that.

#### Convert `account_id` to integer

In [21]:
to_int_udf = F.udf(dfc.make_int, IntegerType())

In [22]:
account_num_col = total_comics_sold['account_num']

In [23]:
total_comics_sold = total_comics_sold.withColumn('account_id'
                                        ,to_int_udf(account_num_col))
total_comics_sold.persist()

DataFrame[account_num: string, comic_title: string, bought: int, account_id: int]

In [24]:
total_comics_sold.show(10)

+-----------+--------------------+------+----------+
|account_num|         comic_title|bought|account_id|
+-----------+--------------------+------+----------+
|      02247|Bubblegun VOL 2 (...|     1|      2247|
|      00487|Captain Swing (Av...|     1|       487|
|      00029|God Is Dead (Avatar)|     1|        29|
|      01260| Providence (Avatar)|     1|      1260|
|      00172|   Supergod (Avatar)|     1|       172|
|      02493|       Abbott (Boom)|     1|      2493|
|      00052|Adventure Time Ma...|     1|        52|
|      00032|Big Trouble In Li...|     1|        32|
|      01149| Broken World (Boom)|     1|      1149|
|      01489|Jim Henson Labyri...|     1|      1489|
+-----------+--------------------+------+----------+
only showing top 10 rows



In [25]:
print(total_comics_sold.count(), len(total_comics_sold.columns))

61871 4


Now I need to find a way to give ids to the `comic_title`. Kind of clunky, but I do have the version in PostgreSQL of the big table. I can just build an ID table up there as source of truth. I could do something on PySpark side, but then think would want to save it somewhere (e.g. the DB) anyway. So might as well do it from the top.

#### Get `comic_id`

In [28]:
comics = spark.read.json('raw_data/comics.json')
comics.persist()

DataFrame[comic_id: bigint, comic_title: string]

In [29]:
comics.count()

7202

In [30]:
comics.show(10)

+--------+--------------------+
|comic_id|         comic_title|
+--------+--------------------+
|       1|0Secret Wars (Mar...|
|       2|100 Bullets Broth...|
|       3|100 Penny Press L...|
|       4|100 Penny Press S...|
|       5|100 Penny Press T...|
|       6|100 Penny Press T...|
|       7|100th Anniversary...|
|       8|12 Reasons To Die...|
|       9|    13 Coins (Other)|
|      10|13th Artifact One...|
+--------+--------------------+
only showing top 10 rows



In [31]:
print(comics.count(), len(comics.columns))

7202 2


Now we need to join this back into `total_comics_sold`.

In [32]:
# Set aliases
tot = total_comics_sold.alias('tot')
com = comics.alias('com')

In [33]:
tot_sold_ids_only = tot.join(com.select('comic_id','comic_title')
                      ,tot.comic_title==com.comic_title).select('account_id'
                                                                , 'comic_id'
                                                                , 'bought')
tot_sold_ids_only.persist()
tot_sold_ids_only.show(10)

+----------+--------+------+
|account_id|comic_id|bought|
+----------+--------+------+
|      2247|     995|     1|
|       487|    1102|     1|
|        29|    2680|     1|
|      1260|    4870|     1|
|       172|    6023|     1|
|      2493|      66|     1|
|        52|     116|     1|
|        32|     755|     1|
|      1149|     971|     1|
|      1489|    3503|     1|
+----------+--------+------+
only showing top 10 rows



In [34]:
tot_sold_ids_only.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- comic_id: long (nullable = true)
 |-- bought: integer (nullable = false)



In [35]:
print(tot_sold_ids_only.count(), len(tot_sold_ids_only.columns))

61871 3


## Can we save this 'clean' df?

In [36]:
als_input_df = tot_sold_ids_only.toPandas()

In [37]:
als_input_df.shape

(61871, 3)

In [38]:
als_input_df.to_json('raw_data/als_input_filtered.json', orient='records'
                     ,lines=True)

In [39]:
!head raw_data/als_input_filtered.json

{"account_id":2247,"comic_id":995,"bought":1}
{"account_id":487,"comic_id":1102,"bought":1}
{"account_id":29,"comic_id":2680,"bought":1}
{"account_id":1260,"comic_id":4870,"bought":1}
{"account_id":172,"comic_id":6023,"bought":1}
{"account_id":2493,"comic_id":66,"bought":1}
{"account_id":52,"comic_id":116,"bought":1}
{"account_id":32,"comic_id":755,"bought":1}
{"account_id":1149,"comic_id":971,"bought":1}
{"account_id":1489,"comic_id":3503,"bought":1}


### ALS Model

Let's start with  train/test split.

In [40]:
# Split data into training and test set
(train, test) = tot_sold_ids_only.randomSplit([.8, .2], seed=41916)

Make sure shapes make sense.

In [41]:
print(train.count(), len(train.columns))

49536 3


In [42]:
print(test.count(), len(test.columns))

12335 3


#### 2nd Run @ ALS with filtered dataset

- Minimum number of titles bought by account = 5
- Maximum number of titles bought by account = 250

In [43]:
now = time.ctime(int(time.time()))
print("Started on {}.".format(now))

Started on Thu Jun 27 21:05:30 2019.


In [68]:
# Create ALS instance and fit model
als = ALS(maxIter=20,
          rank=10,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          alpha=1.0,
          implicitPrefs=True,
          seed=41916)
model = als.fit(train)

In [69]:
now = time.ctime(int(time.time()))
print("Completed on {}.".format(now))

Completed on Thu Jun 27 21:12:52 2019.


### Evaluation on Test

In [46]:
# Generate predictions on TEST
predictions = model.transform(test)
predictions.persist()

DataFrame[account_id: int, comic_id: bigint, bought: int, prediction: float]

In [47]:
predictions.show(10)

+----------+--------+------+------------+
|account_id|comic_id|bought|  prediction|
+----------+--------+------+------------+
|       133|     471|     1|  0.12044321|
|      1489|     496|     1|0.0011770985|
|       579|     496|     1| 0.006886015|
|      2770|     833|     1|0.0036284635|
|      2573|     833|     1|   0.0761981|
|      1621|    1959|     1|  0.09269879|
|      1748|    1959|     1| 0.015198524|
|      1632|    1959|     1|  0.15759571|
|       986|    1959|     1|  0.10084282|
|        49|    1959|     1| 0.008000695|
+----------+--------+------+------------+
only showing top 10 rows



`BinaryClassificationEvaluator` only likes doubles for `rawPredictionCol`, so cast it.

In [48]:
predictions = predictions.withColumn("prediction", predictions["prediction"].cast(DoubleType()))

In [49]:
predictions.show(10)

+----------+--------+------+--------------------+
|account_id|comic_id|bought|          prediction|
+----------+--------+------+--------------------+
|       133|     471|     1| 0.12044321000576019|
|      1489|     496|     1|0.001177098485641...|
|       579|     496|     1|0.006886015180498362|
|      2770|     833|     1|0.003628463484346...|
|      2573|     833|     1| 0.07619810104370117|
|      1621|    1959|     1| 0.09269879013299942|
|      1748|    1959|     1|0.015198524110019207|
|      1632|    1959|     1|  0.1575957089662552|
|       986|    1959|     1| 0.10084281861782074|
|        49|    1959|     1|0.008000695146620274|
+----------+--------+------+--------------------+
only showing top 10 rows



### Initial Evaluation

Based on our first swing of the bat:
- `maxIter` = 20
- `rank` = 10

In [50]:
evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC'
                                          ,labelCol='bought'
                                          ,rawPredictionCol='prediction'
                                          )

In [51]:
auc = evaluator.evaluate(predictions)

In [52]:
print("Area Under the Curve = " + str(auc))

Area Under the Curve = 1.0


#### **REALLY?** 
Smells weird. Especially when we look at the actual `prediction` values.

## Check nans

In [53]:
# Convert to pandas dataframe
pred_df = predictions.select('*').toPandas()

# Check nulls
pred_num_nulls = pred_df['prediction'].isna().sum()

# Num rows
preds_attempted = pred_df.shape[0]

In [54]:
print("There are {} nulls out of {}.".format(pred_num_nulls, preds_attempted))

There are 360 nulls out of 12335.


In [55]:
print("So {:.2f}% are nulls.".format(pred_num_nulls/preds_attempted))

So 0.03% are nulls.


Since, they're such a tiny portion of the test set, for now let's remove the nulls for now.

In [56]:
predictions.count()

12335

In [57]:
# Convert back to spark dataframe
predictions = spark.createDataFrame(pred_df)

In [58]:
predictions.select([count(when(isnan(c), c)).alias(c) for c in predictions.columns]).show()

+----------+--------+------+----------+
|account_id|comic_id|bought|prediction|
+----------+--------+------+----------+
|         0|       0|     0|       360|
+----------+--------+------+----------+



In [59]:
pred_no_na = predictions.na.drop()

In [60]:
pred_no_na.persist()

DataFrame[account_id: bigint, comic_id: bigint, bought: bigint, prediction: double]

In [61]:
pred_no_na.count()

11975

In [62]:
# Evaluate the model by computing the AUC on the test data
auc2 = evaluator.evaluate(pred_no_na)

print("AUC = " + str(auc2))

AUC = 1.0


Ok, so that gives us a baseline. Let's see if we can do a little grid search action.

Before we do that, some testing shows that I can't get the BinaryEvaluator to work because I need to manually cast the `predictionCol` to double, but can't do that while encapsulated inside of 

In [63]:
# Evaluate the model by computing the RMSE on the test data
eval_reg = RegressionEvaluator(metricName="rmse", labelCol="bought",
                                predictionCol="prediction")

### Grid Search

In [ ]:
# Create a parameter grid
params = (ParamGridBuilder()
          .addGrid(als.regParam, [1, 0.01, 0.001, 0.1])
          .addGrid(als.maxIter, [5, 10, 20])
          .addGrid(als.rank, [4, 10, 50])
          .addGrid(als.alpha, [.01, 0.1, 1, 10, 40])).build()

In [ ]:
cv = TrainValidationSplit(estimator=als
                          , evaluator=evaluator
                          , estimatorParamMaps=params
                          , trainRatio=0.8)
#cv = CrossValidator(estimator=als_implicit, estimatorParamMaps=paramGrid, evaluator=RegressionEvaluator())
model_implicit = cv.fit(train)

In [ ]:
### Cross validate for best hyperparameters
cv = CrossValidator(estimator=als
                    ,estimatorParamMaps=params
                    ,evaluator=evaluator
                    ,parallelism=4
                   )

In [ ]:
now = time.ctime(int(time.time()))
print("Completed on {}.".format(now))

In [ ]:
# Fit and store model
best_model = cv.fit(test)

In [ ]:
als_model = best_model.bestModel

In [ ]:
now = time.ctime(int(time.time()))
print("Completed on {}.".format(now))

### Get Top N recommendations for Single User

Let's make a reference list of `account_id`'s, for testing purposes.

In [64]:
n_to_test = 5

users = (tot_sold_ids_only.select(als.getUserCol())
                          .sample(False
                                  ,n_to_test/tot_sold_ids_only.count()
                                  ,41916)
        )
users.persist()
users.show()

+----------+
|account_id|
+----------+
|      2185|
|      1663|
|         5|
|       775|
|       604|
+----------+



We developed and wrote the functionality out to a function in `comic_recs.py`

###  Testing function!

- Pass the function to a pandas dataframe. 
- Function will ask for an account_id.
- Will return top n, n defined in parameters.

In [65]:
top_n_df = cr.get_top_n_recs_for_user(spark=spark, model=model, topn=5)
top_n_df

2185


,comic_title
1,Gideon Falls (Image)
2,Ad After Death Book 01 (of 3 (Image)
3,Ad After Death Book 02 (of 3 (Image)
4,Royal City (Image)
5,Ad After Death Book 03 (of 3 (Image)


In [66]:
top_n_df = cr.get_top_n_recs_for_user(spark=spark, model=model, topn=5)
top_n_df

1663


,comic_title
1,Darth Vader (Marvel)
2,Star Wars Vader Down (Marvel)
3,Star Wars Annual (Marvel)
4,Star Wars (Marvel)
5,Journey Star Wars Fase (Marvel)


In [67]:
top_n_df = cr.get_top_n_recs_for_user(spark=spark, model=model, topn=5)
top_n_df

161


,comic_title
1,Star Wars (Marvel)
2,Star Wars Vader Down (Marvel)
3,Darth Vader (Marvel)
4,Star Wars Annual (Marvel)
5,East of West (Image)


## Conclusions
- OK this seems more realistic(?) Only three tests, but it seems more realistic that two users would completely different recommendations, than other results I've seen to date, where there would be overlap of 2 or 3.

---